In [1]:
# import libaries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
import matplotlib.pyplot as plt
from scipy.stats import poisson

### Step 1: Get EPL Team Overall Stats from web
Get EPL teams' stats using database in www.pesmaster.com.
The result dictionary will have 6 seasons' stats from 13/14 ~ 18/19 EPL.

In [20]:
# list of the teams (18/19)
# data from three different sources use different names for teams. 
# manually making a dictionary to rename teams
team_dict = {
            'Arsenal FC': 'Arsenal',
            'Burnley FC': 'Burnley',
            'Liverpool FC': 'Liverpool',
            'Everton FC': 'Everton',
            'Fulham FC': 'Fulham',
            'Southampton FC': 'Southampton',
            'Chelsea FC': 'Chelsea',
            'West Bromwich Albion': 'West Bromwich',
            'Manchester United': 'Manchester Utd',
            'Manchester City': 'Manchester City', 
            'Newcastle United': 'Newcastle Utd',
            'West Ham United': 'West Ham Utd', 
            'Tottenham Hotspur': 'Tottenham',
            'Queens Park Rangers': 'QP Rangers',
            'Watford FC': 'Watford',
            'AFC Bournemouth': 'Bournemouth',
            'Sunderland AFC': 'Sunderland',
            'Middlesbrough FC': 'Middlesbrough',
            'Brighton & Hove Albion': 'Brighton & Hove',
            'Huddersfield Town': 'Huddersfield',
            'QPR': 'QP Rangers'
            }

In [21]:
teams_seasons = []
for year in range(2014, 2020):
    url = 'https://www.pesmaster.com/english-league/pes-' + str(year) + '/league/9/'
    res = requests.get(url, headers={'User-agent': 'slsl'})
    
    if res.status_code != 200:
        print('Status not 200', res.status_code)
        break

    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find("table", {"id" : "search-result-table"})
    tr = table.find_all('tr')
    
    teams = []

    for i in range(1,(len(tr))):
            result = {}

            td = tr[i].find_all('td')
            
            if td:
                result['Team'] = td[0].text
                result['Ovr'] = td[1].text
                result['Def'] = td[2].text
                result['Mid'] = td[3].text
                result['Fwd'] = td[4].text
                result['Phy'] = td[5].text
                result['Spd'] = (td[6].text)

            if len(result) == 7:
                teams.append(result)
                
    time.sleep(3)
    
    teams_df = pd.DataFrame(teams, columns=(['Team', 'Ovr', 'Def', 'Mid', 'Fwd', 'Phy', 'Spd']))
    teams_df = teams_df.set_index('Team')
    teams_df.rename(index=team_dict, inplace=True)

    # put all seasons into a list
    teams_seasons.append(teams_df)
    
# make a list as a dictionary that has keys = season (2013, 2014, ..., 2017)
teams_dict = {str(2014 + i) : teams_seasons[i] for i in range(0,6)}

In [22]:
teams_dict['2014'].head()

,Ovr,Def,Mid,Fwd,Phy,Spd
Team,,,,,,
Chelsea,86,87,85,87,81,78
Manchester City,86,85,86,92,82,79
Manchester Utd,85,83,85,89,79,78
Arsenal,84,84,85,82,78,80
Tottenham,83,80,83,84,80,79


In [23]:
for season in range(2014, 2020):
    teams_dict[str(season)].to_csv('./Data/PES' + str(season-2000) + '.csv')

In [ ]:
# list of the teams (18/19)
pes_team_dict = {
            'ARSENAL': 'Arsenal',
            'LIVERPOOL': 'Liverpool',
            'SOUTH WALES': 'Cardiff City',
            'MAN BLUE': 'Manchester City',
            'EAST DORSETSHIRE': 'Bournemouth',
            'WEST LONDON WHITE': 'Fulham',
            'EAST MIDLANDS': 'Leicester City',
            'SOUTH NORWOOD': 'Crystal Palace',
            'TYNESIDE': 'Newcastle Utd',
            'NORTH EAST LONDON': 'Tottenham',
            'EAST LONDON': 'West Ham Utd',
            'HAMPSHIRE RED': 'Southampton',
            'LANCASHIRE CLARET': 'Burnley',
            'HERTFORDSHIRE': 'Watford',
            'MAN RED': 'Manchester Utd',
            'EAST SUSSEX': 'Brighton & Hove',
            'LONDON FC': 'Chelsea',
            'MERSEYSIDE BLUE': 'Everton',
            'WM GOLD':'Wolverhampton',
            'WEST YORKSHIRE TOWN': 'Huddersfield',
            'WEST GLAMORGAN CITY': 'Swansea City'}

In [ ]:
players = []

for page in range(1,18):
    url = 'http://pesdb.net/pes2019/?league=1&sort=club_team&order=a&page=' + str(page)
    res = requests.get(url, headers={'User-agent': 'slsl'})
    
    if res.status_code != 200:
        print('Status not 200', res.status_code)
        break

    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find("table", {"class" : "players"})
    tr = table.find_all('tr')

    for i in range(1,(len(tr))):
            result = {}

            td = tr[i].find_all('td')
            
            if td:
                result['Name'] = td[1].text
                result['Team'] = td[2].text
                result['Rating'] = (td[9].text)

            if len(result) == 3:
                players.append(result)
                
    time.sleep(3)
    
players_df = pd.DataFrame(players, columns=(['Name', 'Team', 'Rating']))
final_player = players_df.set_index('Team')
final_player.rename(index=pes_team_dict, inplace=True)


In [ ]:
final_player.head()

### Step 2: Get EPL Tables of Last 5 Seasons (2013~2018)

In [6]:
season_tables = []

for year in range(2014, 2019):
    # load data from soccerstats.com by years (2013 will mean 13/14)
    table_url = 'https://www.soccerstats.com/widetable.asp?league=england_' + str(year)
    res = requests.get(table_url, headers={'User-agent': 'slsl'})

    if res.status_code != 200:
        print('Status not 200', res.status_code)
        break

    soup = BeautifulSoup(res.content, 'lxml')
    
    # get a table
    table = soup.find('table', {'id': 'btable'})
    tr = table.find_all('tr', {'class': 'trow8'})

    final_table = []

    for i in range(0,(len(tr))):
        result = {}

        td = tr[i].find_all('td')

        if td:
            result['Team'] = td[1].text.strip()     # team name
            result['P'] = int(td[2].text)           # number of games
            result['W'] = int(td[3].text)           # wins
            result['D'] = int(td[4].text)           # draws
            result['L'] = int(td[5].text)           # loses
            result['HW'] = int(td[13].text)         # home wins
            result['HGF'] = int(td[16].text)        # home goals scored
            result['HGA'] = int(td[17].text)        # home goals conceded
            result['AGF'] = int(td[27].text)        # away goals scored
            result['AGA'] = int(td[28].text)        # away goals conceded
            result['AW'] = int(td[24].text)         # away wins
            result['GF'] = int(td[6].text)          # goals for
            result['GA'] = int(td[7].text)          # goals allowed
            result['PTS'] = int(td[9].text)         # total points end of the season

            # put them into one dictionary
            if len(result) == 14:
                final_table.append(result)

    time.sleep(3)

    # put dictionary into a pandas dataframe with column names
    final_table = pd.DataFrame(final_table, columns=(['Team', 'P', 'W', 'D', 'L', 'GF', 'GA', 'HW', 'HGF', 'HGA', 'AW', 'AGF', 'AGA', 'PTS']))
    
    # Create features to generate poisson distribution
    final_table['H_Att'] = final_table['HGF'] / 19 / (final_table['HGF'].sum() / 380)
    final_table['A_Att'] = final_table['AGF'] / 19 / (final_table['HGA'].sum() / 380)
    final_table['H_Def'] = final_table['HGA'] / 19 / (final_table['HGA'].sum() / 380) #* table_13['DIS'] * table_13['CS'] / 38
    final_table['A_Def'] = final_table['AGA'] / 19 / (final_table['HGF'].sum() / 380) #* table_13['DIS'] * table_13['CS'] / 38
    
    # change index as a team names
    final_table = final_table.set_index('Team')
    # put all seasons into a list
    season_tables.append(final_table)
  
# make a list as a dictionary that has keys = season (2013, 2014, ..., 2017)
dict_seasons = {str(2013 + i) : season_tables[i] for i in range(0,5)}

In [19]:
dict_seasons['2014'] # EPL Season 2013/2014

,P,W,D,L,GF,GA,HW,HGF,HGA,AW,AGF,AGA,PTS,H_Att,A_Att,H_Def,A_Def
Team,,,,,,,,,,,,,,,,,
Chelsea,38,26,9,3,73,32,15,36,9,11,37,23,87,1.285714,1.783133,0.433735,0.821429
Manchester City,38,24,7,7,83,38,14,44,14,10,39,24,79,1.571429,1.879518,0.674699,0.857143
Arsenal,38,22,9,7,71,36,12,41,14,10,30,22,75,1.464286,1.445783,0.674699,0.785714
Manchester Utd,38,20,10,8,62,37,14,41,15,6,21,22,70,1.464286,1.012048,0.722892,0.785714
Tottenham,38,19,7,12,58,53,10,31,24,9,27,29,64,1.107143,1.301205,1.156627,1.035714
Liverpool,38,18,8,12,52,48,10,30,20,8,22,28,62,1.071429,1.060241,0.963855,1.000000
Southampton,38,18,6,14,54,33,11,37,13,7,17,20,60,1.321429,0.819277,0.626506,0.714286
Swansea City,38,16,8,14,46,49,9,27,22,7,19,27,56,0.964286,0.915663,1.060241,0.964286
Stoke City,38,15,9,14,48,45,10,32,22,5,16,23,54,1.142857,0.771084,1.060241,0.821429


### Step 3: Getting Cleans Sheets

In [8]:
cs_tables = []
seasons = ['2013-14', '2014-15', '2015-16', '2016-17', '2017-18']
for year in seasons:
    #load data from soccerstats.com by years (2013 will mean 13/14)
    table_url = 'https://www.sportsmole.co.uk/football/premier-league/' + year + '/best-defence.html'
    res = requests.get(table_url, headers={'User-agent': 'slee'})
   
    if res.status_code != 200:
        print('Status not 200', res.status_code)
        break
        
    soup = BeautifulSoup(res.content, 'lxml')

    # get a table
    table = soup.find('table', {'class':'leaguetable full'})
    tr = table.find_all('tr')
    cs_table = []

    for i in range(1,(len(tr))):
        result = {}

        td = tr[i].find_all('td')

        if td:
            result['Team'] = td[1].text.strip()          # goals allowed
            result['CS'] = int(td[4].text)         # total points end of the season

            # put them into one dictionary
            if len(result) == 2:
                cs_table.append(result)

    time.sleep(3)

    # put dictionary into a pandas dataframe with column names
    cs_table = pd.DataFrame(cs_table, columns=(['Team', 'CS']))
    # change index as a team names
    cs_table = cs_table.set_index('Team')
    cs_table.rename(index=team_dict, inplace=True)
    # put all seasons into a list
    cs_tables.append(cs_table)

# make a list as a dictionary that has keys = season (2013, 2014, ..., 2017)
dict_cs = {str(2013 + i) : cs_tables[i] for i in range(0,5)}

In [9]:
dict_cs['2013'].head()

,CS
Team,
Chelsea,18
Manchester City,16
Everton,15
Arsenal,17
Manchester Utd,13


### Step 4: Getting 'Discipline' Points

In [10]:
dis_tables = []

for year in range(2013, 2018):
    # load data from transfermarkt.co.uk by years (2013 means 13/14)
    table_url = 'https://www.transfermarkt.co.uk/premier-league/fairnesstabelle/wettbewerb/GB1/saison_id/' + str(year) + '/plus/1'
    res = requests.get(table_url, headers={'User-agent': 'slsl'})

    if res.status_code != 200:
        print('Status not 200', res.status_code)

    soup = BeautifulSoup(res.content, 'lxml')

    # get a table
    table = soup.find('table', {'class': 'items'})
    tbody = table.find('tbody')
    tr = tbody.find_all('tr')

    dis_table = []

    for i in range(0,(len(tr))):
        result = {}

        td = tr[i].find_all('td')

        if td:
            result['Team'] = td[2].text.strip()     # team name
            result['P'] = int(td[3].text)           # games played
            result['YC'] = int(td[5].text)          # total number of yellow cards during the season
            result['RC'] = int(td[8].text)          # red cards
            # DIS = (0.5 * YC + 2 * RC) / P
            # higher 'DIS', the team is more likely to have cards during a match
            result['DIS'] = (int(td[5].text) * 0.5 + int(td[8].text) * 2) / int(td[3].text)    

            # put them into one dictionary
            if len(result) == 5:
                dis_table.append(result)

    time.sleep(3)

    # put dictionary into a pandas dataframe with column names
    dis_table = pd.DataFrame(dis_table, columns=(['Team', 'P', 'YC', 'RC', 'DIS']))
    # change index as a team names
    dis_table = dis_table.set_index('Team')
    dis_table.rename(index=team_dict, inplace=True)
    # put all seasons into a list
    dis_tables.append(dis_table)
    
# make a list as a dictionary that has keys = season (2013, 2014, ..., 2017)
dict_dis = {str(2013 + i) : dis_tables[i] for i in range(0,5)}

In [11]:
dict_dis['2013'].head()

,P,YC,RC,DIS
Team,,,,
Cardiff City,38,49,1,0.697368
Liverpool,38,53,1,0.750000
Everton,38,54,1,0.763158
Fulham,38,56,1,0.789474
Southampton,38,60,0,0.789474


### Step 4: Merge Standing and Discipline

In [12]:
# merge dict_seasons, dict_dis
temp_pd = []
for year in range(2013, 2018):
    temp_pd.append(pd.merge(dict_seasons[str(year)], dict_dis[str(year)][['YC', 'RC', 'DIS']], left_index=True, right_index=True, how='outer'))
dict_past_seasons = {str(2013 + i) : temp_pd[i] for i in range(0,5)}

new_temp = []

for year in range(2013, 2018):
    new_temp.append(pd.merge(dict_past_seasons[str(year)], dict_cs[str(year)][['CS']], left_index=True, right_index=True, how='outer'))

dict_final_seasons = {str(2013 + i) : new_temp[i] for i in range(0,5)}



In [13]:
dict_final_seasons['2013'].head()

,P,W,D,L,GF,GA,HW,HGF,HGA,AW,...,AGA,PTS,H_Att,A_Att,H_Def,A_Def,YC,RC,DIS,CS
Team,,,,,,,,,,,,,,,,,,,,,
Arsenal,38,24,7,7,68,41,13,36,11,11,...,30,79,1.204013,1.409692,0.484581,1.003344,51,4,0.881579,17
Aston Villa,38,10,8,20,39,61,6,22,29,4,...,32,38,0.735786,0.748899,1.277533,1.070234,78,0,1.026316,9
Cardiff City,38,7,9,22,32,74,5,20,35,2,...,39,30,0.668896,0.528634,1.541850,1.304348,49,1,0.697368,7
Chelsea,38,25,7,6,71,27,15,43,11,10,...,16,82,1.438127,1.233480,0.484581,0.535117,54,3,0.868421,18
Crystal Palace,38,13,6,19,33,48,8,18,23,5,...,25,45,0.602007,0.660793,1.013216,0.836120,59,2,0.881579,12


In [14]:
for season in range(2013, 2018):
    dict_final_seasons[str(season)].to_csv('./Data/epl_table_' + str(season-2000) + str(season-1999) + '.csv')

### Step 5: Create a Scoring Model (Poisson Distribution)

In [15]:
def score_percentage(dataframe, hometeam, awayteam):
    home_avg = dataframe['HGF'].sum()/380
    away_avg = dataframe['HGA'].sum()/380
    
    home_score = float(dataframe[dataframe.index == hometeam]['H_Att']) * float(dataframe[dataframe.index == awayteam]['A_Def']) * home_avg
    away_score = float(dataframe[dataframe.index == awayteam]['A_Att']) * float(dataframe[dataframe.index == hometeam]['H_Def']) * away_avg

    score = []
    # maximum score for a team is 5
    for goals in range(0, 6):
        scores = {}
        scores['Home'] = (poisson.pmf(goals, home_score) * 100) # Hometeam score
        scores['Away'] = (poisson.pmf(goals, away_score) * 100) # away score

        if len(scores) == 2:
                    score.append(scores)
    score = pd.DataFrame(score, columns=(['Home', 'Away']))
    home_w = 0
    away_w = 0
    draw = 0
    for home in range(1, len(score)):
        for away in range(0, home):
            home_w += (score['Home'][home] * score['Away'][away] / 10000)

    for away in range(1, len(score)):
        for home in range(0, away):
            away_w += (score['Home'][home] * score['Away'][away] / 10000)

    for home in range(0, len(score)):
        away = home
        draw += (score['Home'][home] * score['Away'][away] / 10000)
        
    return home_w, away_w, draw


In [16]:
for years in range(2013, 2018):
    for home_team in dict_final_seasons[str(years)].index:
        tot_hw = 0
        tot_aw = 0
        tot_dr = 0
        count = 0
        for against_team in dict_final_seasons[str(years)].index:
            if against_team != home_team:
                home_w, away_w, draw = score_percentage(dict_final_seasons[str(years)], home_team, against_team)
                tot_hw += home_w
                tot_aw += away_w
                tot_dr += draw
                dict_final_seasons[str(years)].loc[dict_final_seasons[str(years)].index == home_team, 'HWPR'] = tot_hw / 19
                dict_final_seasons[str(years)].loc[dict_final_seasons[str(years)].index == home_team, 'HLPR'] = tot_aw / 19
                dict_final_seasons[str(years)].loc[dict_final_seasons[str(years)].index == home_team, 'HDPR'] = tot_dr / 19

    for away_team in dict_final_seasons[str(years)].index:
        tot_hw = 0
        tot_aw = 0
        tot_dr = 0
        count2 = 0
        for home_team in dict_final_seasons[str(years)].index:
            if home_team != against_team:
                home_w, away_w, draw = score_percentage(dict_final_seasons[str(years)], home_team, against_team)
                tot_hw += home_w
                tot_aw += away_w
                tot_dr += draw
                dict_final_seasons[str(years)].loc[dict_final_seasons[str(years)].index == against_team, 'AWPR'] = tot_hw / 19
                dict_final_seasons[str(years)].loc[dict_final_seasons[str(years)].index == against_team, 'ALPR'] = tot_aw / 19
                dict_final_seasons[str(years)].loc[dict_final_seasons[str(years)].index == against_team, 'ADPR'] = tot_dr / 19
print(count, count2)

0 0


In [17]:
dict_final_seasons['2013'].T

Team,Arsenal,Aston Villa,Cardiff City,Chelsea,Crystal Palace,Everton,Fulham,Hull City,Liverpool,Manchester City,Manchester Utd,Newcastle Utd,Norwich City,Southampton,Stoke City,Sunderland,Swansea City,Tottenham,West Bromwich,West Ham Utd
P,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000
W,24.000000,10.000000,7.000000,25.000000,13.000000,21.000000,9.000000,10.000000,26.000000,27.000000,19.000000,15.000000,8.000000,15.000000,13.000000,10.000000,11.000000,21.000000,7.000000,11.000000
D,7.000000,8.000000,9.000000,7.000000,6.000000,9.000000,5.000000,7.000000,6.000000,5.000000,7.000000,4.000000,9.000000,11.000000,11.000000,8.000000,9.000000,6.000000,15.000000,7.000000
L,7.000000,20.000000,22.000000,6.000000,19.000000,8.000000,24.000000,21.000000,6.000000,6.000000,12.000000,19.000000,21.000000,12.000000,14.000000,20.000000,18.000000,11.000000,16.000000,20.000000
GF,68.000000,39.000000,32.000000,71.000000,33.000000,61.000000,40.000000,38.000000,101.000000,102.000000,64.000000,43.000000,28.000000,54.000000,45.000000,41.000000,54.000000,55.000000,43.000000,40.000000
GA,41.000000,61.000000,74.000000,27.000000,48.000000,39.000000,85.000000,53.000000,50.000000,37.000000,43.000000,59.000000,62.000000,46.000000,52.000000,60.000000,54.000000,51.000000,59.000000,51.000000
HW,13.000000,6.000000,5.000000,15.000000,8.000000,13.000000,5.000000,7.000000,16.000000,17.000000,9.000000,8.000000,6.000000,8.000000,10.000000,5.000000,6.000000,11.000000,4.000000,7.000000
HGF,36.000000,22.000000,20.000000,43.000000,18.000000,38.000000,24.000000,20.000000,53.000000,63.000000,29.000000,23.000000,17.000000,32.000000,27.000000,21.000000,33.000000,30.000000,24.000000,25.000000
HGA,11.000000,29.000000,35.000000,11.000000,23.000000,19.000000,38.000000,21.000000,18.000000,13.000000,21.000000,28.000000,18.000000,23.000000,17.000000,27.000000,26.000000,23.000000,27.000000,26.000000
AW,11.000000,4.000000,2.000000,10.000000,5.000000,8.000000,4.000000,3.000000,10.000000,10.000000,10.000000,7.000000,2.000000,7.000000,3.000000,5.000000,5.000000,10.000000,3.000000,4.000000


In [18]:
dict_final_seasons['2013'].sort_values(by='AWPR', ascending=False)[['AW', 'PTS']]

,AW,PTS
Team,,
West Ham Utd,4,40
Arsenal,11,79
Aston Villa,4,38
Cardiff City,2,30
Chelsea,10,82
Crystal Palace,5,45
Everton,8,72
Fulham,4,32
Hull City,3,37
